In [ ]:
!pip3 install textblob

In [1]:
import warnings
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import keras
import os
import re

from textblob import TextBlob, Word
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet


from sklearn import random_projection
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import OneHotEncoder

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten, Reshape
from keras.layers import Conv1D, Conv2D, Convolution2D, MaxPool2D, MaxPooling2D, Flatten, Reshape, BatchNormalization, Concatenate
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model

from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam
from keras.regularizers import l1, l2

# from tensorflow.contrib.cudnn_rnn.python.ops import cudnn_rnn_ops

from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras import metrics
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

stop_words = set(stopwords.words('english') + list(string.punctuation))

warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Data Reading

In [2]:
Train = pd.read_csv("data/train.csv")
Test = pd.read_csv("data/test.csv")
Valid = pd.read_csv("data/valid.csv")

#################################################################
# Train = Train.iloc[0:10000,:]
#################################################################

In [3]:
Train.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,dsvFUqrhytVmKXCW7bKNhA,0,2013-03-10 01:24:07,0,58nqw-MdO6EDACPaKDM59Q,5.0,"As a student, by back and neck are under const...",0,1NImHsg1kc76n_cQyTm0bg
1,6YrO-hJNof4wsx4f0YQ8yg,0,2014-12-22 17:06:23,0,eVeQtMGaB5tdCH0hKJwxKw,3.0,Stayed here for a football game at University ...,0,Oi0G3jFm2jtG2W02dZTdEQ
2,LtyoPfxpvcF_9e9wMoUi0w,0,2016-04-25 09:55:57,0,AHMHUbq0eAUOuOPFoeO5iw,4.0,"Very good salads, generous portions. I either ...",1,4jbz7cOVuV_Q7v2b3pNrLw
3,QzvLnOqwH6BIY_jCOvzuQQ,0,2017-04-06 04:18:48,0,8RDvwneSMsbpmi5UgGq60A,1.0,The experience I had with this company growing...,0,gJgPs0QXE587T9SIR6NCdQ
4,tPaweigPsXacvQT8daYT4g,0,2017-03-22 15:38:59,0,IboCoxoL0IFtJnsGPHCugg,5.0,"Easiest furniture purchase, with a great price...",0,zp8XLBJmgw55ZTlp5raK7Q


In [4]:
# Label
Train_y = Train['stars']
Test_y = Valid['stars']

ohe = OneHotEncoder(sparse=False)
y_ohe_train = ohe.fit_transform(Train_y.values.reshape(-1, 1))
y_ohe_test = ohe.fit_transform(Test_y.values.reshape(-1, 1))

In [5]:
train = Train[['text','stars']]
test = Valid[['text','stars']]

In [6]:
train.head()

,text,stars
0,"As a student, by back and neck are under const...",5.0
1,Stayed here for a football game at University ...,3.0
2,"Very good salads, generous portions. I either ...",4.0
3,The experience I had with this company growing...,1.0
4,"Easiest furniture purchase, with a great price...",5.0


## Basic Features

In [7]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

def features(df):
    ## Number of Words
    df['word_count'] = df['text'].apply(lambda x: len(str(x).split(" "))) 
    ## Number of characters
    df['char_count'] = df['text'].str.len() 
    ## Average Word Length
    df['avg_word'] = df['text'].apply(lambda x: avg_word(x))
    ## Number of stopwords
    df['stopwords'] = df['text'].apply(lambda x: len([x for x in x.split() if x in stop_words]))
    ## Number of special characters
    df['hastags'] = df['text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
    ##Number of numerics
    df['numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
    ## Number of Uppercase words
    df['upper'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [8]:
features(train)
features(test)

In [9]:
train.head()

,text,stars,word_count,char_count,avg_word,stopwords,hastags,numerics,upper
0,"As a student, by back and neck are under const...",5.0,48,295,5.166667,14,0,0,2
1,Stayed here for a football game at University ...,3.0,37,218,4.918919,13,0,0,0
2,"Very good salads, generous portions. I either ...",4.0,19,106,4.631579,4,0,0,2
3,The experience I had with this company growing...,1.0,134,716,4.343284,55,0,2,7
4,"Easiest furniture purchase, with a great price...",5.0,19,122,5.473684,6,0,0,0


## Pre-processing

In [17]:
def pre_processing(df):
    
    ## Lower case
    df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    ## Removing Punctuation
    df['text'] = df['text'].str.replace('[^\w\s]','')
    ## Removing Stopwords
    df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
    ## Spelling correction (time-required)
#     df['text'] = df['text'].apply(lambda x: str(TextBlob(x).correct()))
    

In [18]:
pre_processing(train)
pre_processing(test)

In [19]:
train['text'].head()

0        student back neck constant strain dr serrick m...
1        stayed football game university phoenix stadiu...
2        good salads generous portions either get mexic...
3        experience company growing awesome least memor...
4        easiest furniture purchase great price deliver...
5        great hotel clean dont like pay 34 resort fee ...
6        café coffee actually important worse version p...
7        impressed ever wanting try place finally night...
8        first night vegas wanted something simple dinn...
9        21 reviews 12 1star variety coincidence shock ...
10       place used amazing amazing service amazing foo...
11       best fried chicken ever town concert stumbled ...
12       early bird specials 11am 10am forget soup buns...
13       really like place definitely perfect play area...
14       little disappointed 20oz bonein ribeye 50 came...
15       whoever picked name convenient little restaura...
16       extremely extremely super nice staff owners mo.

### Rare words removal

In [20]:
df = pd.concat([train, test])

In [21]:
freq = pd.Series(' '.join(df['text']).split()).value_counts()[-10:]
freq

lancasters       1
heinekens        1
opsayo           1
tablelike        1
madisoncom       1
mosse            1
hamerschlag      1
stimulatingly    1
businesstype     1
juuuuuuust       1
dtype: int64

In [22]:
freq = list(freq.index)
train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
test['text'] = test['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

## Stemming and Lemmatization

In [23]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def stem_lemm(df):
    train['text'] = train['text'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))
    train['text'] = train['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [24]:
stem_lemm(train)
stem_lemm(test)

## Tokenization

In [25]:
tfidf = TfidfVectorizer(max_features=100, lowercase=True, analyzer='word',
                        stop_words='english', ngram_range=(1, 2))

train_vect = tfidf.fit_transform(train['text'])
test_vect = tfidf.fit_transform(test['text'])


In [36]:
tokenizer = Tokenizer(100)
tokenizer.fit_on_texts(train['text'])
sequences = tokenizer.texts_to_sequences(train['text'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 92161 unique tokens.


In [26]:
# Show the Model as a pandas DataFrame
feature_names = tfidf.get_feature_names()
train_vectdf = pd.DataFrame(train_vect.toarray(), columns = feature_names)
train_vectdf.head()

,amazing,area,asked,awesome,bad,bar,best,better,big,bit,...,vegas,wait,want,wanted,wasnt,way,went,work,worth,youre
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.179496,0.0,0.0,0.0,0.171823
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


## Bag of Words

In [27]:
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(train['text'])

In [ ]:
train_bow.shape

## Glove

In [48]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = '../glove.6B.100d.txt'
word2vec_output_file = '../glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

from gensim.models import KeyedVectors 

filename = '../glove.6B.100d.txt.word2vec'
embeddings_index = KeyedVectors.load_word2vec_format(filename, binary=False)

In [49]:
MAX_NUM_WORDS = 1000
MAX_SEQUENCE_LENGTH = 100

In [50]:
EMBEDDING_DIM = embeddings_index['a'].shape[0]
num_words = 400000
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

In [71]:
for word, i in word_index.items():
    if i > 10:
        continue
    # This references the loaded embeddings dictionary
    embedding_vector = embeddings_index[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

KeyError: "word 'servic' not in vocabulary"

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [88]:
# Hyperparameters
input_length = 100
embedding_size = 100
MAX_SEQUENCE_LENGTH = 30
hidden_size = 100
batch_size = 100
dropout_rate = 0.5
filters = 100
kernel_sizes = [3, 6, 9]
padding = 'valid'
activation = 'relu'
strides = 1
pool_size = 2
learning_rate = 0.1
total_epoch = 10
output_size = 5

In [89]:
# Creating Callbacks
# ModelCheckpoints is used to save the model after every epoch
# EarlyStopping is used to stop training when the validation loss has not improved after 2 epochs
# Tensorboard is used tovisualize dynamic graphs of the training and test metrics
cbks = [callbacks.ModelCheckpoint(filepath='./checkpoint_model.h5', monitor='val_loss', save_best_only=True),
            callbacks.EarlyStopping(monitor='val_loss', patience=2),callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)]

In [93]:
# Final Model Architecture

model = Sequential()
model.add(Embedding(input_dim=100000, output_dim=embedding_size,
                    input_length=input_length, dropout=0.2))
model.add(Conv1D(filters=filters, kernel_size=3, padding='same',
                 activation='relu', activity_regularizer=l1(0.001)))
model.add(Conv1D(filters=filters, kernel_size=6, padding='same',
                 activation='relu', activity_regularizer=l1(0.01)))
model.add(Conv1D(filters=filters, kernel_size=9, padding='same',
                 activation='relu', kernel_regularizer=l2(1.0)))
model.add(MaxPooling1D(pool_size=((input_length-kernel_sizes[2])//strides+1)))
# 1 layer of 100 units in the hidden layers of the LSTM cells
model.add(LSTM(100))
model.add(Dense(output_size, activation='softmax',
                activity_regularizer=l1(0.001)))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 100)          10000000  
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 100, 100)          30100     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 100, 100)          60100     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 100, 100)          90100     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 1, 100)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 505       
Total para

In [ ]:
model.fit(train_vect, y_ohe_train, epochs=10,
          verbose=1, batch_size=32, callbacks=cbks)

train_score = model.evaluate(
    train_vect, y_ohe_train, batch_size=batch_size)
test_score = model.evaluate(
    test_vect, y_ohe_test, batch_size=batch_size)

print('Training Loss: {}\n Training Accuracy: {}\n'
      'Testng Loss: {}\n Testing accuracy: {}'.format(
          train_score[0], train_score[1],
          test_score[0], test_score[1]))

Epoch 1/10
  6976/100000 [=>............................] - ETA: 9:54 - loss: 1.4575 - acc: 0.4438

## Visualization of loss and accuracy

In [96]:
## Visualizing Loss

plt.switch_backend('agg')
%matplotlib inline

fig1 = plt.figure()
plt.plot(model.model['loss'],'r',linewidth=3.0)
plt.legend(['Training Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('CNN Loss',fontsize=16)
fig1.savefig('loss_cnn.png')
plt.show()

TypeError: 'Sequential' object is not subscriptable

<Figure size 432x288 with 0 Axes>

In [ ]:
## Visualizing accuracy

fig2 = plt.figure()
plt.plot(model.model['acc'], 'r', linewidth=3.0)
plt.legend(['Training Accuracy'], fontsize=18)
plt.xlabel('Epochs ', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.title('CNN Accuracy', fontsize=16)
fig2.savefig('accuracy_cnn.png')
plt.show()